# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df
df = pd.read_csv('insurance.csv',sep=',')

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [ ]:
X = df[['age', 'sex', 'bmi', 'children', 'smoker','region_northeast', 'region_northwest', 'region_southeast','region_southwest']]
y = df['charges']

In [ ]:
X

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,0,1,0,0
4,32,1,28.880,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,0,1,0,0
1334,18,0,31.920,0,0,1,0,0,0
1335,18,0,36.850,0,0,0,0,1,0
1336,21,0,25.800,0,0,0,0,0,1


In [ ]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
X_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
43,37,0,30.80,2,0,0,0,1,0
1104,37,1,29.80,0,0,0,0,0,1
1021,22,0,31.02,3,1,0,0,1,0
875,23,0,28.12,0,0,0,1,0,0
425,45,1,24.31,5,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
837,56,0,28.31,0,0,1,0,0,0
759,18,1,38.17,0,1,0,0,1,0
739,29,1,35.50,2,1,0,0,0,1
447,56,0,25.65,0,0,0,1,0,0


In [ ]:
X_test

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
693,24,1,23.655,0,0,0,1,0,0
1286,28,0,17.290,0,0,1,0,0,0
1084,62,0,30.495,2,0,0,1,0,0
921,62,0,33.200,0,0,0,0,0,1
158,30,1,35.530,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...
465,30,0,28.380,1,1,0,0,1,0
530,57,1,42.130,1,1,0,0,1,0
1060,24,1,32.010,0,0,0,0,1,0
213,34,0,26.730,1,0,0,0,1,0


In [ ]:
y_train

43       6313.75900
1104    20420.60465
1021    35595.58980
875      2690.11380
425      9788.86590
           ...     
837     11657.71890
759     36307.79830
739     44585.45587
447     11454.02150
1263     7337.74800
Name: charges, Length: 936, dtype: float64

In [ ]:
y_test

693      2352.96845
1286     3732.62510
1084    15019.76005
921     13462.52000
158     36950.25670
           ...     
465     19521.96820
530     48675.51770
1060     1981.58190
213      5002.78270
471      2203.47185
Name: charges, Length: 402, dtype: float64

## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression() 

In [ ]:
lr.fit(X_train, y_train)

LinearRegression()

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание
lr.coef_

array([  261.50410925,   197.92773293,   339.63450707,   511.42875807,
       23865.43557672,   540.61614005,   455.3875921 ,  -801.23032969,
        -194.77340246])

In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test, lr.predict(X_test))

40425385.32465099

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

34970850.11176225

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([26638.87068557])